# NBA Shot Locations

In this notebook we looked at NBA shot charts over time. We started by getting the shots data for the Raptors using [nba_api](https://github.com/swar/nba_api).

In [ ]:
team_name = 'Raptors'
seasons = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2021-22', '2022-23']

from nba_api.stats.static import teams
from nba_api.stats.endpoints import shotchartdetail
import pandas as pd

team = teams.find_teams_by_full_name(team_name)[0]
team_id = team['id']
shots = pd.DataFrame()
for season in seasons:
    season_shots = shotchartdetail.ShotChartDetail(team_id=team_id, player_id=0, season_nullable=season, season_type_all_star=['Regular Season', 'Playoffs'], context_measure_simple='FGA').get_data_frames()[0]
    season_shots['SEASON'] = season
    shots = pd.concat([shots, season_shots])
shots

There are quite a few interesting columns in the dataset.

In [ ]:
shots.columns

We converted the `SHOT_MADE_FLAG` column to boolean (true or false) values and created a scatterplot of the shots over time to see if there were any obvious patterns.

In [ ]:
shots['SHOT_MADE_FLAG'] = shots['SHOT_MADE_FLAG'].astype(bool)

import plotly.express as px
px.scatter(shots, x='LOC_X', y='LOC_Y', color='SHOT_MADE_FLAG', hover_data='SHOT_DISTANCE', animation_frame='SEASON', height=800, width=800)

Next we wanted to see if there was a relationship between the time remaining in the game and the shot distance. To do that we needed to create a new column that combined `MINUTES_REMAINING` and `SECONDS_REMAINING`. We also converted the `PERIOD` column to a string so that we can select individual periods in the legend.

In [ ]:
shots['TIME_REMAINING'] = shots['MINUTES_REMAINING'] * 60 + shots['SECONDS_REMAINING']
shots['PERIOD'] = shots['PERIOD'].astype(str)
px.scatter(shots, x='TIME_REMAINING', y='SHOT_DISTANCE', color='PERIOD', title='Shot Distance by Time Remaining', hover_data=['SHOT_DISTANCE','SHOT_MADE_FLAG'], height=800)

It does look like the shots from the furthest away were most likely to be right before the end of a period.

We also see a denser band of shots taken close to the basket and near the 3-point line, a histogram of shot distances will help us see that better.

In [ ]:
px.histogram(shots, x='SHOT_DISTANCE', color='SHOT_MADE_FLAG', animation_frame='SEASON', title='Shot Distance Frequencies by Season').update_xaxes(range=[0, 80]).update_yaxes(range=[0, 1000])

## Conclusion

Throughout all of the seasons that we looked at, the Toronto Raptors were more likely to shoot from near the basket and near (but beyond) the 3-point line. We also found that the "long shots" from far away from the basket were more likely to be in the last seconds of a period.

If the future, it would be interesting to compare this to other teams or seasons, or to see if there are correlations such as between `SHOT_TYPE` and `SHOT_DISTANCE`.